# ABOUT
This notebook is meant for translating normal A4 PDF to A3 booklets optimised for printing at TU Delft
* saves paper
* saves money
* dont have to deal with annoying flips in single stapled documents


# Methodology
1. Make booklet
    1. first arrange the PDF and insert the blank pages at the right places: the code hence formed returns a PDF with single pages in the reverse order
    2. after single page booklet-order formation the booklet needs to be formed in A3
2. Rotate pages
    * once the A3 booklet like Adobe booklet is formed, we rotate every even numbered page by 180 degrees so as to make it printable in A3 at TU Delft printers, since the printers only print along the short edge of A3

In [4]:
from PyPDF2 import PdfFileReader, PdfFileWriter
# from PyPDF2 import PyPDF2.PageObject  as PageObject
import os
# from pdfimpose import SaddleImpositor  
# import pdfimpose

## Rotate pages
rotates every second page by 180 degrees for printing at TU delft printers

In [39]:
def rotate_pages(pdf_path):
    """
    Summary: This code takes the pdf path as input and returns rotated pdf file with every even numbered page rotated by 180 degrees
    ___________________
    Params:
    pdf_path (str): path of pdf to be rotated 
    ___________________
    Returns:
    None
    """
    pdf_root_folder = os.path.dirname(pdf_path)
    outname= os.path.split(pdf_path)[1][:-4] # the first [1] is for extracting the trailing part of path, i.e. filename, and then the [:-4] slicing is to get name without the .pdf

    pdf_writer = PdfFileWriter()
    pdf_reader = PdfFileReader(pdf_path)

    pg = pdf_reader.numPages

    # check if exists folder for outpath, make output dir
    outpath = os.path.join(pdf_root_folder, "rotated_output")
    if not os.path.exists(outpath) :
        os.mkdir(outpath)
    
    for page in range(pg): 
        # print(page)
  
        # creating rotated page object 
        pageObj = pdf_reader.getPage(page)
        # rotate if even page, page is odd
        if page%2 != 0:
            # print("here")
            pageObj.rotateClockwise(180) 
            pdf_writer.addPage(pageObj) 
        else:
            # pageObj.rotateClockwise(180) 
            pdf_writer.addPage(pageObj) 

    with open( os.path.join(outpath, outname + '_rot.pdf' ) , 'wb') as fh:
        pdf_writer.write(fh)

# rotate_pages("/mnt/e/TU_Delft/yr_2/q5/CIE4604/Material/te printen/halliday_gravitation.pdf", "hall_rotat.pdf")
# rotate_pages("/mnt/e/TU_Delft/yr_2/q5/CIE4604/Material/te printen/rees_ch10.pdf","rees_rotat")


## Booklet formation

This [Link for theory](https://wiki.scribus.net/canvas/Making_a_booklet_with_Scribus_and_Adobe_Reader) explains that the process for booklet formation is known as imposition

There are SDK's available for imposing PDFs online, however the following code block does the same procedure
<!-- [another link](https://stackoverflow.com/questions/45226318/pdf-imposition-using-python) -->

we will be using the library pdfimpose from pip for this purpose, found [here](https://pypi.org/project/pdfimpose/)

what we want to do is found in the [docs of the library](https://pdfimpose.readthedocs.io/en/latest/library/#module-pdfimpose.schema.saddle)

<!-- # classpdfimpose.schema.saddle.SaddleImpositor(last: int = 0, omargin: pdfimpose.schema.common.Margins = Margins(left=0, right=0, top=0, bottom=0), mark: list = <factory>, folds: Optional[str] = None, imargin: float = 0, bind: str = 'left', creep: Callable[[int], float] = <function nocreep>)[s -->


## My implementation

when I had not read about the word imposition with regards to PDFs and printing, I was plamnning to make my own code for PDF imposition

In [14]:
a3_width = 1190.5511811024
a3_height = 841.8897637795


def make_booklet(pdf_path):
    pdf_writer = PdfFileWriter()
    pdf_reader = PdfFileReader(pdf_path)
    pg = pdf_reader.numPages 
    # understand the different configurations of booklet
    case = pg%4
    print(pg, case, int(pg/2))

    if (case== 0):
        # all divisible
        # begin 1 half from front and 1 half from back
        firsthalf = list(range(int(pg/2)))
        last = list(range(int(pg/2),pg))
        ind  = pg 
        # firsthalf = [0]
        # Generate a mount of A3
        
    
        for _ in firsthalf :
            
            print(f"_ is {_}")
            ind-=1

            if _%2 == 0:
                print("adding blank")
                base_page1 = pdf_writer.addBlankPage(width = a3_width, height = a3_height)
                # even, odd
                print(_+1,'\t',ind+1)
                # get page even
                page_obj = pdf_reader.getPage( ind ) 
                # get page odd 
                page_obj_bk = pdf_reader.getPage( _ ) 
                # place the PDF to the left of the # A3 
                base_page1.mergePage( page_obj )
                # right to place the PDF of # A3
                base_page1.mergeRotatedScaledTranslatedPage(page_obj_bk , 0, 1, a3_width / 2, 0, expand=False)
                print(base_page1)
#                 pdf_writer.addPage(base_page1)
            else:
                
                base_page2 = pdf_writer.addBlankPage(width = a3_width, height = a3_height)
                # odd, even
                print(ind+1,'\t',_+1)
                # get page odd 
                page_obj_bk = pdf_reader.getPage( _ ) 
                
                # get page even
                page_obj = pdf_reader.getPage( ind ) 
                # place the PDF to the left of the # A3 
                base_page2.mergePage( page_obj_bk )
                
                # right to place the PDF of # A3
                base_page2.mergeRotatedScaledTranslatedPage(page_obj , 0, 1, a3_width / 2, 0, expand=False)
#                 pdf_writer.addPage(base_page2)

            
            


    elif(case==1):
        pass

    elif(case==2):
        pass

    elif(case==3):        
        pass

    pdf_output_file = open('test_booklet.pdf','wb')
    pdf_writer.write(pdf_output_file )
    pdf_output_file.close()


make_booklet("/home/purusharth/code/fun_experiments_with_programs/pdf_manipulate/tester.pdf")



20 0 10
_ is 0
adding blank
1 	 20
{'/Type': '/Page', '/Parent': IndirectObject(1, 0), '/Resources': {'/Font': {'/F159': IndirectObject(89, 0), '/F162': IndirectObject(90, 0), '/F167': IndirectObject(91, 0)}, '/XObject': {'/Im6': IndirectObject(483, 0), '/Im1': IndirectObject(76, 0), '/Im2': IndirectObject(78, 0)}, '/ColorSpace': {'/pgfprgb': ['/Pattern', '/DeviceRGB']}, '/ProcSet': ['/PDF', '/Text']}, '/MediaBox': RectangleObject([0, 0, 1190.55118, 841.88976]), '/Contents': {}, '/Annots': [IndirectObject(482, 0), IndirectObject(484, 0), IndirectObject(75, 0), IndirectObject(77, 0), IndirectObject(79, 0), IndirectObject(80, 0), IndirectObject(81, 0), IndirectObject(82, 0), IndirectObject(84, 0)]}
_ is 1
19 	 2
_ is 2
adding blank
3 	 18
{'/Type': '/Page', '/Parent': IndirectObject(1, 0), '/Resources': {'/Font': {'/F159': IndirectObject(89, 0), '/F162': IndirectObject(90, 0), '/F167': IndirectObject(91, 0)}, '/ColorSpace': {'/pgfprgb': ['/Pattern', '/DeviceRGB']}, '/ProcSet': ['/PDF', '

## Rotate in a folder
the following cell takes a folder path, makes pdf paths and then returns a new folder (from the rotation function) with the even pages of all pdf's rotated by 180 degrees

In [40]:
# path = path to the folder containing the pdf files
path = "/mnt/e/TU_Delft/yr_2/q5/CIE4604/Material/te printen/"

j= [x for x in os.listdir(path) if x.endswith(".pdf")]
pathlist = [os.path.join(path, x) for x in j] # list of paths of pdf in folder
outpath= "/mnt/e/TU_Delft/yr_2/q5/CIE4604/Material/te printen/fin_print"
for filepath in pathlist:
    nm = filepath.split('/')[-1]
    rotate_pages(filepath)
    #  print(nm)

here
here
here
here
here
here


here
here
here
here
here
here
here
here


# New approach to Booklet formation / PDF impose
the following cell has code adapted from a fellow stackoverflow user for the purpose of imposing a given PDF as a booklet, as it would have been the case in Adobe Acrobat Booklet formation

In [27]:
pgn = 11
int(math.ceil(10/4))

# math.ceil(2.1)
print(math.ceil(8/4))
print(math.ceil(9/4))
print(math.ceil(10/4))
print(math.ceil(12/4))

_ = 2* (np.arange(3) +1)
_[::-1]

2
3
3
3


array([6, 4, 2])

In [37]:

import sys
import os
import math
import numpy as np
from PyPDF2 import PdfFileWriter as Writer, PdfFileReader as Reader

def main_impose(fname):
    """ Input file name; output a rearanged pdf file in the same folder
    The documentation uses a file with 44 pages as an example"""
    # read PDF
    orig = Reader(open(fname, 'rb'))
    origpages = orig.pages
    # initialise writer for new PDF
    new = Writer()
    
    nop = len(origpages)
    nop_booklet = int(math.ceil(nop / 4.0)) # = 11

    base = np.arange(nop_booklet) + 1
    base = 2 * base
    base = base[::-1] #reverses the list
    pages = [] # list to arrange the page numbers 
    for i in base:
        num = nop_booklet * 4 + 1 # = 45
        pages.append(i)
        pages.append(num - i)
        pages.append(num - i + 1)
        pages.append(num - (num - i + 1))
    # print(pages)

    bookletOrderedPagesTuples = list((pages[i],pages[i+1]) for i in range(0,len(pages),2) )[::-1]
    bookletOrderedPagesList = [item for sublist in bookletOrderedPagesTuples for item in sublist]

    # adding pages to the new PDF
    for i in bookletOrderedPagesList:
        if i > nop:
            new.addBlankPage()
        else:
            idx = int(i - 1)
            new.addPage(origpages[idx])

    # save the modified pdf file
    fn = os.path.join(os.path.dirname(fname), os.path.basename(fname) + ".booklet.pdf")
    with open(fn, 'wb') as f:
        new.write(f)
    print("File saved as {}".format(fn))



In [38]:
main_impose('./saddle_10.pdf')

[6, 7, 8, 5, 4, 9, 10, 3, 2, 11, 12, 1]
File saved as ./saddle_10.pdf.booklet.pdf


In [59]:
l = [6, 7, 8, 5, 4, 9, 10, 3, 2, 11, 12, 1]
j = list((l[i],l[i+1]) for i in range(0,len(l),2) )[::-1]
print(j)
[item for sublist in j for item in sublist]
# j

[(12, 1), (2, 11), (10, 3), (4, 9), (8, 5), (6, 7)]


[12, 1, 2, 11, 10, 3, 4, 9, 8, 5, 6, 7]